In [1]:
import ee

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [3]:
from flood_detection import modis
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio as rio
import geopandas as gpd
from dateutil import rrule
from datetime import datetime
import rioxarray as rioxr

# specify these

Example of shapefile: [Honduras Shapefile](https://data.humdata.org/dataset/cod-ab-hnd?)

Put it in `Data/Shapefiles/`, and unzip it

In [4]:
shapefilePath = Path('Data/Shapefiles/hnd_admbnda_adm0_sinit_20161005.shp')

In [5]:
if not shapefilePath.exists():
    print("Shapefile not found!!!")
else:
    print("Shapefile found, good to go")

Shapefile found, good to go


### This is the output path where the rasters will go

In [6]:
outputPath = Path('Data/Rasters/GFD/') / shapefilePath.stem
outputPath.mkdir(exist_ok=True, parents=True)

### Start and end date of what you want to generate, with time resolution (dt)

In [7]:
startDate = '2020-01-01'
endDate = '2020-01-13'
dt = rrule.WEEKLY # either YEARLY, MONTHLY, or WEEKLY

### Options for the GFD algorithm

see [GFD Github Page](https://github.com/cloudtostreet/MODIS_GlobalFloodDatabase) for more details

In [8]:
thresholdType = 'standard' # "standard" or "otsu"
compositionDays = '3Day' # "2Day" or "3Day". The DFO algorithm uses multiple days of images to remove false detections from cloud shadows.
slopeThreshold = 5 # applies a mask to remove pixels that are greater than a certain slope. Set to -1 if you do not want to apply a slope threshold

# Prepare the data

In [9]:
# read shapefile
shapefile = gpd.read_file(shapefilePath)

In [10]:
# upload convex hull to gee (shapefile itself is usually too complex to be uploaded)
gdpFrame = gpd.GeoDataFrame(shapefile.convex_hull)
gdpFrame.columns = ['geom']
gdpFrame = gdpFrame.set_geometry('geom')
gdpFrame = gdpFrame.set_crs(shapefile.crs)
eeShape = geemap.geopandas_to_ee(gdpFrame).geometry()

In [11]:
# create vector of date to iterate over
dates = list(map(lambda date: date.strftime('%Y-%m-%d'), rrule.rrule(dt,
                      dtstart=datetime.strptime(startDate, '%Y-%m-%d'),
                      until=datetime.strptime(endDate, '%Y-%m-%d'))))

In [12]:
# specify delta time in ee format
ee_dt = 'Year'
if dt == rrule.MONTHLY:
    ee_dt = 'Month'
if dt == rrule.WEEKLY:
    ee_dt = 'Week'

# Download Data

In [13]:
def DownloadImageForDate(date, ee_dt, geom):
    
    # calculate start and end date
    startDate = ee.Date(date)
    endDate = ee.Date(date).advance(1, ee_dt)
    endDateString = endDate.advance(-1, 'day').format("YYYY-MM-dd").getInfo()

    # specify file name
    filename = outputPath/f"{date}_{endDateString}.tif"
    if filename.exists():
        return

    # create the floodmap code on ee
    flood_map = modis.dfo(geom, startDate, endDate, thresholdType, compositionDays, True)
    
    # apply slope mask if threshold is larger than -1
    if slopeThreshold > -1:
        flood_map = misc.apply_slope_mask(flood_map, thresh=slopeThreshold)
    
    # add permanent water as band
    perm_water = misc.get_jrc_perm(geom)
    
    #combine flood maps
    dfo_final = ee.Image(flood_map).addBands(perm_water)
    
    # download
    geemap.download_ee_image(dfo_final, filename, crs='EPSG:4326', region=geom, scale=250)

In [14]:
# iterate over dates and download
for k, date in enumerate(dates):
    print(f"Downloading {k+1}/{len(dates)}")
    DownloadImageForDate(date, ee_dt, eeShape)

Collected and pre-processed MODIS Images
DFO Flood Dectection Complete


2020-01-01_2020-01-07.tif: |    | 0.00/106M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


Collected and pre-processed MODIS Images
DFO Flood Dectection Complete


2020-01-08_2020-01-14.tif: |    | 0.00/106M (raw) [  0.0%] in 00:00 (eta:     ?)

# Postprocess the files

In [15]:
## add band descriptions to files

for file in outputPath.glob('**/*.tif'):
    with rio.open(file,'r+') as dst:
        dst.descriptions = tuple(['flooded', 'duration', 'clearViews', 'clearPerc', 'maxExtent', 'permanentWater'])

In [16]:
## Clip to geometry

for file in outputPath.glob('**/*.tif'):
    with rioxr.open_rasterio(file) as dst:
        data = dst
    data.rio.clip(shapefile.geometry).rio.to_raster(file)